## Imports

In [1]:
import csv
from pymongo import MongoClient

import nltk
from nltk.tokenize import word_tokenize
from nltk.util import ngrams

from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem import PorterStemmer
from nltk.stem.lancaster import LancasterStemmer

import pickle

import pandas as pd

import re
import string

from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.preprocessing import scale
from sklearn.cluster import KMeans
from sklearn.utils import shuffle

# sets backend to render higher res images
%config InlineBackend.figure_formats = ['retina']

# see https://ipython.readthedocs.io/en/stable/interactive/magics.html
%pylab inline

Populating the interactive namespace from numpy and matplotlib


/Users/farsyd/anaconda3/envs/metis/lib/python3.6/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['shuffle']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [2]:
# nltk.download('popular')

## Converting CSV to List of Dicts

In [3]:
# #Only when loading to MongoDB first time
# filename = 'data/Nita_Lake_Lodge_Whistler-Whistler_British_Columbia__en.csv'

In [4]:
# #Only when loading to MongoDB first time
# with open(filename) as f:
#     data_listofdict = [{k: v for k, v in row.items()}
#         for row in csv.DictReader(f, skipinitialspace=True)]

In [5]:
# #Only when loading to MongoDB first time
# data_listofdict[0]

## Connect to MongoDB on AWS Instance

In [6]:
config = {
  'host': '18.188.138.161:27017',
  'username': 'mongo_user',
  'password': 'password',
  'authSource': 'cool'
}

In [7]:
client = MongoClient(**config)

ta_db = client.ta_reviews

In [8]:
# secure_config = {
#   'host': '18.188.138.161:27017',
#   'username': 'admin',
#   'password': 'secure_password',
#   'authSource': 'cool'
# }

In [9]:
# admin_client = MongoClient(**secure_config)

In [10]:
# ta_db = admin_client["ta_reviews"]

In [11]:
# use_config = {
#   'host': '18.188.138.161:27017',
#   'username': 'admin',
#   'password': 'secure_password',
#   'authSource': 'ta_reviews'
# }

In [12]:
# use_client = MongoClient(**use_config)

# use_db = use_client.ta_reviews

## Insert list of dicts into MongoDB

In [13]:
## Uncomment when first inserting data into MongoDB
# ta_db.nita.insert_many(data_listofdict)

The list of dictionaries has been inserted into MongoDB

## Explore Data

In [14]:
# When initially running notebook only
ta_obj = ta_db.nita

In [15]:
# # For initial exploration
# ta_db.list_collection_names()

In [16]:
# # For initial exploration
# ta_db.nita.count()

In [17]:
# list(ta_obj.find().limit(1))

## NLP Pipeline

### Defining the Class

In [18]:
# class nlp_preprocessor:
   
#     def __init__(self, vectorizer=CountVectorizer(), tokenizer=None, cleaning_function=None, 
#                  stemmer=None, model=None):
#         """
#         A class for pipelining our data in NLP problems. The user provides a series of 
#         tools, and this class manages all of the training, transforming, and modification
#         of the text data.
#         ---
#         Inputs:
#         vectorizer: the model to use for vectorization of text data
#         tokenizer: The tokenizer to use, if none defaults to split on spaces
#         cleaning_function: how to clean the data, if None, defaults to the in built class
#         """
#         if not tokenizer:
#             tokenizer = self.splitter
#         if not cleaning_function:
#             cleaning_function = self.clean_text
#         self.stemmer = stemmer
#         self.tokenizer = tokenizer
#         self.model = model
#         self.cleaning_function = cleaning_function
#         self.vectorizer = vectorizer
#         self._is_fit = False
        
#     def splitter(self, text):
#         """
#         Default tokenizer that splits on spaces naively
#         """
#         return text.split(' ')
        
#     def clean_text(self, text, tokenizer, stemmer):
#         """
#         A naive function to lowercase all works can clean them quickly.
#         This is the default behavior if no other cleaning function is specified
#         """
#         cleaned_text = []
#         for post in text:
#             cleaned_words = []
#             for word in tokenizer(post):
#                 low_word = word.lower()
#                 if stemmer:
#                     low_word = stemmer.stem(low_word)
#                 cleaned_words.append(low_word)
#             cleaned_text.append(' '.join(cleaned_words))
#         return cleaned_text
    
#     def fit(self, text):
#         """
#         Cleans the data and then fits the vectorizer with
#         the user provided text
#         """
#         clean_text = self.cleaning_function(text, self.tokenizer, self.stemmer)
#         self.vectorizer.fit(clean_text)
#         self._is_fit = True
        
#     def transform(self, text):
#         """
#         Cleans any provided data and then transforms the data into
#         a vectorized format based on the fit function. Returns the
#         vectorized form of the data.
#         """
#         if not self._is_fit:
#             raise ValueError("Must fit the models before transforming!")
#         clean_text = self.cleaning_function(text, self.tokenizer, self.stemmer)
#         return self.vectorizer.transform(clean_text)
    
#     def save_pipe(self, filename):
#         """
#         Writes the attributes of the pipeline to a file
#         allowing a pipeline to be loaded later with the
#         pre-trained pieces in place.
#         """
#         if type(filename) != str:
#             raise TypeError("filename must be a string")
#         pickle.dump(self.__dict__, open(filename+".mdl",'wb'))
        
#     def load_pipe(self, filename):
#         """
#         Writes the attributes of the pipeline to a file
#         allowing a pipeline to be loaded later with the
#         pre-trained pieces in place.
#         """
#         if type(filename) != str:
#             raise TypeError("filename must be a string")
#         if filename[-4:] != '.mdl':
#             filename += '.mdl'
#         self.__dict__ = pickle.load(open(filename,'rb'))

### NLP Functions

## Get Review Text from MongoDB into Pandas DataFrame

In [19]:
# cursor = ta_obj.find({'helpful_vote' : '0'}, {'_id':0, 'review_body': 1}).limit(3)

# list(cursor)

In [20]:
#When notebook runs initially
ta_df = pd.DataFrame(list(ta_obj.find()))

In [21]:
#When notebook runs initially
ta_df.head(15)

,_id,helpful_vote,rating,review_body,review_date
0,5d5d83cac56427b284d5398f,23,50,One of best and most reasonably priced places ...,"August 13, 2019"
1,5d5d83cac56427b284d53990,38,50,We spent 3 nights at the hotel- it was really ...,"July 7, 2019"
2,5d5d83cac56427b284d53991,4,40,"While there were a few bumps, this is a lovely...","August 11, 2019"
3,5d5d83cac56427b284d53992,27,40,Very pleased with front desk staff and Geoff (...,"August 11, 2019"
4,5d5d83cac56427b284d53993,1,50,Nita Lake Lodge is a wonderful choice if you a...,"August 10, 2019"
5,5d5d83cac56427b284d53994,0,50,We had a wonderful stay at Nita Lake Lodge. Ou...,"August 10, 2019"
6,5d5d83cac56427b284d53995,2,50,Everything about staying here was excellent. F...,"August 7, 2019"
7,5d5d83cac56427b284d53996,1,50,This is by far my home away from home! Never d...,"August 2, 2019"
8,5d5d83cac56427b284d53997,0,50,We spent three nights at the lodge for a work-...,"July 31, 2019"
9,5d5d83cac56427b284d53998,3,50,This review is long overdue. We visited the lo...,"July 30, 2019"


In [22]:
# reviews = ta_df[['_id','review_body']]
# reviews.head()

## Old Preprocessing

### Remove Punctuation

In [23]:
# # Replace punctuations with a white space
# reviews['review_body'] = reviews['review_body'].str.replace('[^\w\s]','')

### Make all text lowercase

In [24]:
# reviews['review_body'] = reviews['review_body'].str.lower()

### Remove Numbers

In [25]:
# reviews['review_body'].str.replace('\w*\d\w*','')

### Remove non-English

In [26]:
# words = set(nltk.corpus.words.words())

In [27]:
# reviews['review_body'] = reviews['review_body'].str.replace('[^A-Za-z0-9]', ' ')

### Remove Hotel Name

In [28]:
# reviews['review_body'] = reviews['review_body'].str.replace('nita lake lodge', ' ')

In [29]:
# reviews.head()

### Remove unhelpful words

In [30]:
# remove_words = ['nita']

In [31]:
# remove_words = [
#     'nice', 'didnt', 'did'
#     ,'great', 'really', 'like'
#     ,'good', 'lovely', 'beautiful'
#     ,'just', 'right', 'like'
#     ,'hotel', 'amazing', 'excellent'
#     ,'little', 'nita'
# ]

In [32]:

# reviews['review_body'] = reviews['review_body'].apply(lambda x: ' '.join([word for word in x.split() if word not in (remove_words)]))



In [33]:
# reviews.head()

### Stemmer

In [34]:
# stemmer = PorterStemmer()

In [35]:
# reviews['review_body'] = reviews['review_body'].apply(lambda x: ' '.join([stemmer.stem(word) for word in x.split()]))

### Tokenization (Words)

In [36]:
# reviews['review_body'] = reviews['review_body'].apply(word_tokenize)

In [37]:
# reviews.head()

### Vectorization with Count Vectorizer

In [38]:
# cv = CountVectorizer(stop_words='english', ngram_range=(1,1), 
#                      max_df = 1.0, min_df = 1)

# X1 = cv.fit_transform(reviews['review_body'])
# temp = pd.DataFrame(X1.toarray(), columns=cv.get_feature_names())

# type(X1)


# count_vec = pd.concat([reviews, temp], axis = 1)

# count_vec.set_index('_id', inplace = True)

# count_vec.head()

### Vectorization with TF-IDF

In [39]:
# tfidf1 = TfidfVectorizer(stop_words='english', ngram_range=(2, 3)
#                         , max_df = 1.0, min_df = 0.01)
# X2 = tfidf1.fit_transform(reviews['review_body'])

# temp2 = pd.DataFrame(X2.toarray(), columns=tfidf1.get_feature_names())
# temp2.head()

# tf_idf = pd.concat([reviews, temp2], axis = 1)
# tf_idf.set_index('_id', inplace = True)
# tf_idf.head()

## K-Means Clustering

In [40]:
# inertia = []
# list_num_clusters = list(range(1,7))
# for num_clusters in list_num_clusters:
#     km = KMeans(n_clusters=num_clusters)
#     km.fit(X1)
#     inertia.append(km.inertia_)
    
# plt.plot(list_num_clusters,inertia)
# plt.scatter(list_num_clusters,inertia)
# plt.xlabel('Number of Clusters')
# plt.ylabel('Inertia');

## Preprocessing with Spacy

### Spacy Setup

In [41]:
import spacy #load spacy
nlp = spacy.load("en", disable=['parser', 'tagger', 'ner'])

spacy_stopwords = spacy.lang.en.stop_words.STOP_WORDS
nltk_stopwords = nltk.corpus.stopwords.words('english')

In [42]:
def normalize(comment, lowercase, remove_stopwords):
    if lowercase:
        comment = comment.lower()
    comment = nlp(comment)
    lemmatized = list()
    for word in comment:
        lemma = word.lemma_.strip()
        if lemma:
            if not remove_stopwords or (remove_stopwords and lemma not in spacy_stopwords):
                lemmatized.append(lemma)
    return " ".join(lemmatized)

In [43]:
reviews_spacy = ta_df[['_id','review_body']]
reviews_spacy.head()

,_id,review_body
0,5d5d83cac56427b284d5398f,One of best and most reasonably priced places ...
1,5d5d83cac56427b284d53990,We spent 3 nights at the hotel- it was really ...
2,5d5d83cac56427b284d53991,"While there were a few bumps, this is a lovely..."
3,5d5d83cac56427b284d53992,Very pleased with front desk staff and Geoff (...
4,5d5d83cac56427b284d53993,Nita Lake Lodge is a wonderful choice if you a...


### Remove Hotel Name, Make Lowercase

In [44]:
# Remove Hotel Name
# Need to make lowercase otherwise hotel name in caps is not removed. Ugghhh!!
reviews_spacy['review_body'] = reviews_spacy['review_body'].str.lower()
reviews_spacy['review_body'] = reviews_spacy['review_body'].str.replace('nita lake lodge', ' ')
reviews_spacy.head()

/Users/farsyd/anaconda3/envs/metis/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/farsyd/anaconda3/envs/metis/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


,_id,review_body
0,5d5d83cac56427b284d5398f,one of best and most reasonably priced places ...
1,5d5d83cac56427b284d53990,we spent 3 nights at the hotel- it was really ...
2,5d5d83cac56427b284d53991,"while there were a few bumps, this is a lovely..."
3,5d5d83cac56427b284d53992,very pleased with front desk staff and geoff (...
4,5d5d83cac56427b284d53993,is a wonderful choice if you are looking to ...


### Remove Punctuation

In [45]:
# Replace punctuations with a white space
reviews_spacy['review_body'] = reviews_spacy['review_body'].str.replace('[^\w\s]','')
reviews_spacy.head()

/Users/farsyd/anaconda3/envs/metis/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,_id,review_body
0,5d5d83cac56427b284d5398f,one of best and most reasonably priced places ...
1,5d5d83cac56427b284d53990,we spent 3 nights at the hotel it was really e...
2,5d5d83cac56427b284d53991,while there were a few bumps this is a lovely ...
3,5d5d83cac56427b284d53992,very pleased with front desk staff and geoff a...
4,5d5d83cac56427b284d53993,is a wonderful choice if you are looking to ...


### Running Spacy

In [46]:
reviews_spacy['After_Clean'] = reviews_spacy['review_body'].apply(normalize, lowercase=True, remove_stopwords=True)
reviews_spacy.head()

/Users/farsyd/anaconda3/envs/metis/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,_id,review_body,After_Clean
0,5d5d83cac56427b284d5398f,one of best and most reasonably priced places ...,reasonably price place stay canada amaze servi...
1,5d5d83cac56427b284d53990,we spent 3 nights at the hotel it was really e...,spend 3 night hotel enjoyable impress location...
2,5d5d83cac56427b284d53991,while there were a few bumps this is a lovely ...,bump lovely property early tell -PRON- room re...
3,5d5d83cac56427b284d53992,very pleased with front desk staff and geoff a...,desk staff geoff australian look bike kayak ge...
4,5d5d83cac56427b284d53993,is a wonderful choice if you are looking to ...,wonderful choice look away beautiful surroundi...


In [47]:
# Make Final DF
spacy_final = reviews_spacy[["_id", "After_Clean"]]
spacy_final.head()

,_id,After_Clean
0,5d5d83cac56427b284d5398f,reasonably price place stay canada amaze servi...
1,5d5d83cac56427b284d53990,spend 3 night hotel enjoyable impress location...
2,5d5d83cac56427b284d53991,bump lovely property early tell -PRON- room re...
3,5d5d83cac56427b284d53992,desk staff geoff australian look bike kayak ge...
4,5d5d83cac56427b284d53993,wonderful choice look away beautiful surroundi...


### Removing Non-English Chars

In [48]:
# Remove Non English chars
spacy_final['After_Clean'] = spacy_final['After_Clean'].str.replace('[^A-Za-z0-9]', ' ')

/Users/farsyd/anaconda3/envs/metis/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


### Remove Unhelpful Words

In [49]:
# remove_words = ['nita']

remove_words = [
    'nice', 'didnt', 'did'
    ,'great', 'really', 'like'
    ,'good', 'lovely', 'beautiful'
    ,'just', 'right', 'like', 'PRON'
    ,'hotel', 'amazing', 'excellent'
    ,'little', 'nita', 'tell', 'come'
]

In [50]:

spacy_final['After_Clean'] = spacy_final['After_Clean'].apply(
    lambda x: ' '.join([word for word in x.split() if word not in (remove_words)]))

spacy_final.head(20)


/Users/farsyd/anaconda3/envs/metis/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,_id,After_Clean
0,5d5d83cac56427b284d5398f,reasonably price place stay canada amaze servi...
1,5d5d83cac56427b284d53990,spend 3 night enjoyable impress location away ...
2,5d5d83cac56427b284d53991,bump property early room ready happen desk new...
3,5d5d83cac56427b284d53992,desk staff geoff australian look bike kayak ge...
4,5d5d83cac56427b284d53993,wonderful choice look away surrounding lodge s...
5,5d5d83cac56427b284d53994,wonderful stay room appoint view lake room cle...
6,5d5d83cac56427b284d53995,stay ease check walk lake suit large restauran...
7,5d5d83cac56427b284d53996,far home away home disappoint love perk drive ...
8,5d5d83cac56427b284d53997,spend night lodge workrelated event workshop r...
9,5d5d83cac56427b284d53998,review long overdue visit lodge couple year ag...


### Count Vectorize Spacy

In [51]:
cv_spacy = CountVectorizer(ngram_range=(2,3), min_df = 0.01)

In [52]:
X3 = cv_spacy.fit_transform(spacy_final['After_Clean'])
temp3 = pd.DataFrame(X3.toarray(), columns=cv_spacy.get_feature_names())

In [53]:
count_vec_spacy = pd.concat([spacy_final, temp3], axis = 1)
count_vec_spacy.set_index('_id', inplace = True)
count_vec_spacy.head()

,After_Clean,10 minute,20 night,30 minute,amaze view,aura restaurant,away hustle,away hustle bustle,away village,away whistler,...,whistler blackcomb,whistler creekside,whistler mountain,whistler stay,whistler time,whistler village,wife stay,wish stay,wish stay long,wonderful stay
_id,,,,,,,,,,,,,,,,,,,,,
5d5d83cac56427b284d5398f,reasonably price place stay canada amaze servi...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5d5d83cac56427b284d53990,spend 3 night enjoyable impress location away ...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
5d5d83cac56427b284d53991,bump property early room ready happen desk new...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5d5d83cac56427b284d53992,desk staff geoff australian look bike kayak ge...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5d5d83cac56427b284d53993,wonderful choice look away surrounding lodge s...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### TF-IDF Spacy

In [54]:
tfidf_spacy = TfidfVectorizer(ngram_range=(2, 3), min_df=0.01)

In [55]:
X4 = tfidf_spacy.fit_transform(spacy_final['After_Clean'])
temp4 = pd.DataFrame(X4.toarray(), columns=tfidf_spacy.get_feature_names())

In [56]:
tf_idf_spacy = pd.concat([spacy_final, temp4], axis = 1)
tf_idf_spacy.set_index('_id', inplace = True)
tf_idf_spacy.head()

,After_Clean,10 minute,20 night,30 minute,amaze view,aura restaurant,away hustle,away hustle bustle,away village,away whistler,...,whistler blackcomb,whistler creekside,whistler mountain,whistler stay,whistler time,whistler village,wife stay,wish stay,wish stay long,wonderful stay
_id,,,,,,,,,,,,,,,,,,,,,
5d5d83cac56427b284d5398f,reasonably price place stay canada amaze servi...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
5d5d83cac56427b284d53990,spend 3 night enjoyable impress location away ...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.228178,0.0,0.0,0.0,0.0
5d5d83cac56427b284d53991,bump property early room ready happen desk new...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
5d5d83cac56427b284d53992,desk staff geoff australian look bike kayak ge...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
5d5d83cac56427b284d53993,wonderful choice look away surrounding lodge s...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0


## Pickling all Vectors

### For LSA and NMF

#### Count Vectorized Data

In [57]:
# with open('count_vec.pickle', 'wb') as to_write:
#     pickle.dump(count_vec, to_write)

#### TF-IDF'd Data

In [58]:
# with open('tf_idf.pickle', 'wb') as to_write:
#     pickle.dump(tf_idf, to_write)

#### Count Vectorized Data with Spacy

In [59]:
with open('count_vec_spacy.pickle', 'wb') as to_write:
    pickle.dump(count_vec_spacy, to_write)

#### TF-IDF'd Data with Spacy

In [60]:
with open('tf_idf_spacy.pickle', 'wb') as to_write:
    pickle.dump(tf_idf_spacy, to_write)

### Pickling for LDA

#### Count Vectorized Data

In [61]:
# # Pickling the transposed sparse matrix for LDA
# X_cv = X1.T
# # X_cv.shape

# id2word_cv = dict((v, k) for k, v in cv.vocabulary_.items())

# with open('X_cv.pickle', 'wb') as to_write:
#     pickle.dump(X_cv, to_write)
    
# with open('id2word_cv.pickle', 'wb') as to_write:
#     pickle.dump(id2word_cv, to_write)

#### TF-IDF'd Data

In [62]:
# # Pickling the transposed sparse matrix for LDA
# X_tfi = X2.T
# # X_cv.shape

# id2word_tfi = dict((v, k) for k, v in tfidf1.vocabulary_.items())

# with open('X_tfi.pickle', 'wb') as to_write:
#     pickle.dump(X_tfi, to_write)
    
# with open('id2word_tfi.pickle', 'wb') as to_write:
#     pickle.dump(id2word_tfi, to_write)

#### Count Vectorized Data with Spacy

In [63]:
# Pickling the transposed sparse matrix for LDA
X_cvspacy = X3.T
# X_cv.shape

id2word_cvspacy = dict((v, k) for k, v in cv_spacy.vocabulary_.items())

with open('X_cvspacy.pickle', 'wb') as to_write:
    pickle.dump(X_cvspacy, to_write)
    
with open('id2word_cvspacy.pickle', 'wb') as to_write:
    pickle.dump(id2word_cvspacy, to_write)

#### TF-IDF'd Data with Spacy

In [64]:
# Pickling the transposed sparse matrix for LDA
X_tfispacy = X4.T
# X_cv.shape

id2word_tfispacy = dict((v, k) for k, v in tfidf_spacy.vocabulary_.items())

with open('X_tfispacy.pickle', 'wb') as to_write:
    pickle.dump(X_tfispacy, to_write)
    
with open('id2word_tfispacy.pickle', 'wb') as to_write:
    pickle.dump(id2word_tfispacy, to_write)

#### For Regression

In [66]:
y = ta_df[['helpful_vote']]
y.head()

,helpful_vote
0,23
1,38
2,4
3,27
4,1


In [67]:
y.shape

(2145, 1)

In [68]:
with open('y.pickle', 'wb') as to_write:
    pickle.dump(y, to_write)